# Combining Python, SQL DBs and AWS

## Aim: Create a RDB instance on AWS, connect to it through python, and execute queries.  


## Create an instance on AWS  
    
https://aws.amazon.com/getting-started/tutorials/create-mysql-db/

##  Using MYSQLWorkbench

Before we complicate things with using Python, we want to use MYSQLWorkbench to make sure we set up our AWS server correctly and can connect to it.  

- Create a connection to the AWS server
- Execute a simple create table query
- Drop the table


# Using Python with MYSQL DB

In [17]:
## importing 'mysql.connector' 
import mysql.connector

In [18]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector .connect(
    host = "localhost",
    user = "root",
    passwd = "dbms"
)

DatabaseError: 2003 (HY000): Can't connect to MySQL server on 'localhost' (61)

### Documentation:

https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html

## Using a config File:

You do not want to make your credentials viewable to everyone who might see this file.  So instead of explicity stating your credentials, we want to import them from another file.  

In [22]:
!pip freeze

alabaster==0.7.12
anaconda-client==1.7.2
anaconda-navigator==1.9.7
anaconda-project==0.8.2
appnope==0.1.0
appscript==1.0.1
asn1crypto==0.24.0
astroid==2.2.5
astropy==3.1.2
atomicwrites==1.3.0
attrs==19.1.0
Babel==2.6.0
backcall==0.1.0
backports.os==0.1.1
backports.shutil-get-terminal-size==1.0.0
beautifulsoup4==4.7.1
bitarray==0.8.3
bkcharts==0.2
bleach==3.1.0
bokeh==1.0.4
boto==2.49.0
Bottleneck==1.2.1
certifi==2019.3.9
cffi==1.12.2
chardet==3.0.4
Click==7.0
cloudpickle==0.8.0
clyent==1.2.2
colorama==0.4.1
conda==4.7.11
conda-build==3.17.8
conda-package-handling==1.3.11
conda-verify==3.1.1
config==0.4.2
contextlib2==0.5.5
cryptography==2.6.1
cycler==0.10.0
Cython==0.29.6
cytoolz==0.9.0.1
dask==1.1.4
decorator==4.4.0
defusedxml==0.5.0
distributed==1.26.0
docutils==0.14
entrypoints==0.3
et-xmlfile==1.0.1
fastcache==1.0.2
filelock==3.0.10
Flask==1.0.2
future==0.17.1
gevent==1.4.0
glob2==0.6
gmpy2==2.0.8
greenlet==0.4.15
h5py==2.9.0
heapdict==1.0.0
html5lib==1.0.1
idna==2.8
imageio==2.5.0

In [23]:
import config2

In [24]:
#config.pw_example

In [26]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    password = config2.passwd
)

print(cnx) # it will print a connection object if everything is fine


In [27]:
cursor = cnx.cursor()

## Creating DB

In [28]:
import mysql.connector
from mysql.connector import errorcode

In [29]:
db_name = 'employees'

In [30]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

Database employees does not exists.
Database employees created successfully.


### Creating tables

In [31]:
DB_NAME = 'employees'

TABLES = {}
TABLES['employees'] = (
    "CREATE TABLE employees ("
    "  emp_no int(11) NOT NULL AUTO_INCREMENT,"
    "  birth_date date NOT NULL,"
    "  first_name varchar(14) NOT NULL,"
    "  last_name varchar(16) NOT NULL,"
    "  gender enum('M','F') NOT NULL,"
    "  hire_date date NOT NULL,"
    "  PRIMARY KEY (emp_no)"
    ") ENGINE=InnoDB")

TABLES['departments'] = (
    "CREATE TABLE departments ("
    "  dept_no char(4) NOT NULL,"
    "  dept_name varchar(40) NOT NULL,"
    "  PRIMARY KEY (dept_no), UNIQUE KEY dept_name (dept_name)"
    ") ENGINE=InnoDB")

TABLES['salaries'] = (
    "CREATE TABLE salaries ("
    "  emp_no int(11) NOT NULL,"
    "  salary int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT salaries_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_emp'] = (
    "CREATE TABLE dept_emp ("
    "  emp_no int(11) NOT NULL,"
    "  dept_no char(4) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no), KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_emp_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_emp_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_manager'] = (
    "  CREATE TABLE dept_manager ("
    "  dept_no char(4) NOT NULL,"
    "  emp_no int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no),"
    "  KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_manager_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_manager_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['titles'] = (
    "CREATE TABLE titles ("
    "  emp_no int(11) NOT NULL,"
    "  title varchar(50) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date DEFAULT NULL,"
    "  PRIMARY KEY (emp_no,title,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT titles_ibfk_1 FOREIGN KEY (emp_no)"
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

In [32]:
TABLES['employees']

"CREATE TABLE employees (  emp_no int(11) NOT NULL AUTO_INCREMENT,  birth_date date NOT NULL,  first_name varchar(14) NOT NULL,  last_name varchar(16) NOT NULL,  gender enum('M','F') NOT NULL,  hire_date date NOT NULL,  PRIMARY KEY (emp_no)) ENGINE=InnoDB"

In [34]:
cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    passwd = config2.passwd,
    database = DB_NAME
)
cursor = cnx.cursor()

In [35]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table employees: OK
Creating table departments: OK
Creating table salaries: OK
Creating table dept_emp: OK
Creating table dept_manager: OK
Creating table titles: OK


## Inserting Data

In [36]:
from datetime import date, datetime, timedelta

#since we closed the connection, we need to reinstatiate it
cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    passwd = config2.passwd,
    database = DB_NAME
)
cursor = cnx.cursor()

tomorrow = datetime.now().date() + timedelta(days=1)

add_employee = ("INSERT INTO employees "
               "(first_name, last_name, hire_date, gender, birth_date) "
               "VALUES (%s, %s, %s, %s, %s)")
add_salary = ("INSERT INTO salaries "
              "(emp_no, salary, from_date, to_date) "
              "VALUES (%(emp_no)s, %(salary)s, %(from_date)s, %(to_date)s)")

data_employee = ('Geert', 'Vanderkelen', tomorrow, 'M', date(1977, 6, 14))



In [37]:
# Insert new employee
cursor.execute(add_employee, data_employee)
emp_no = cursor.lastrowid

# Insert salary information
data_salary = {
  'emp_no': emp_no,
  'salary': 50000,
  'from_date': tomorrow,
  'to_date': date(9999, 1, 1),
}
cursor.execute(add_salary, data_salary)

# Make sure data is committed to the database
cnx.commit()

cursor.close()
cnx.close()

## Insert Many

cursor.executemany(operation, seq_of_params)

In [38]:
cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    passwd = config2.passwd,
    database = DB_NAME
)
cursor = cnx.cursor()

In [39]:
data = [
  ('Jane', date(2005, 2, 12)),
  ('Joe', date(2006, 5, 23)),
  ('John', date(2010, 10, 3)),
]
stmt = "INSERT INTO employees (first_name, hire_date) VALUES (%s, %s)"
cursor.executemany(stmt, data)

## Quyerying the DB

In [40]:
cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    passwd = config2.passwd,
    database = DB_NAME
)
cursor = cnx.cursor()

In [42]:

query = ("SELECT first_name, last_name, hire_date FROM employees "
         "WHERE hire_date BETWEEN %s AND %s")

hire_start = datetime(1999, 1, 1)
hire_end = datetime(1999, 12, 31)

cursor.execute(query, (hire_start, hire_end))

for (first_name, last_name, hire_date) in cursor:
  print("{}, {} was hired on {:%d %b %Y}".format(
    last_name, first_name, hire_date))

cursor.close()
cnx.close()


# Picking up from yesterday

## Select Statements

Select statements is the primary type of query you will write, as this is how we write q query to pull data back from a db


https://www.codecademy.com/articles/sql-commands

### SELECT
SELECT chooses the fields that you want displayed in your chart. This is the specific piece of information that you want to pull from your database.

**Syntax**

`SELECT first_name, last_name;`

Select * will pull back every column in the table


 ### FROM
FROM pinpoints the table that you want to pull the data from. 

**Syntax** 
`SELECT first_name, last_name
 FROM table;`

### WHERE
WHERE allows you to filter your query to be more specific. You  set up a conditional to filter your data

**syntax**

`SELECT first_name, last_name
FROM table
WHERE column = value`

### AND
AND allows you to add additional criteria to your WHERE statement. 


**Syntax**

`SELECT
     first_name, last_name
FROM
     table
WHERE
     column = value
AND
     column BETWEEN value AND value;`


### ORDER BY

Your ORDER BY clause will allow you to sort by any of the fields that you have specified in the SELECT statement. 

**Syntax**

`SELECT
     first_name, last_name
FROM
     table
WHERE
     column = value
AND
     column BETWEEN value AND value
ORDER BY
     column;`






### GROUP BY
"GROUP BY" is similar to the function in pandas where it will aggregate data that has similarities.

Here is your SQL query:

**Syntax**

`SELECT
     first_name, COUNT(last_name)
FROM
     table
WHERE
     column = value
AND
     column BETWEEN value AND value
GROUP BY
     column;`




### LIMIT
Depending on the amount of data you have in your database, it may take a long time to run the queries. It can be frustrating if you find yourself waiting a long time to run a query that you didn't really want to begin with. If you want to test our query, the LIMIT function is a great one to use because it allows you to limit the number of results you get.

**Syntax**

`SELECT
     first_name, COUNT(last_name)
FROM
     table
WHERE
     column = value
AND
     column BETWEEN value AND value
GROUP BY
     column
LIMIT
     100;`

## Applied: Working with our student data

### Import our student data

In [43]:
import json

f=open('students.json','r')
data=json.load(f)

### Create a connection and cursor to AWS db

In [56]:
# 
cnx = mysql.connector.connect(
        host = config2.host,
        user = config2.user,
        passwd = config2.passwd,
        database= DB_NAME)
cursor = cnx.cursor()

### Create a table for our student info

In [57]:

create_query = """
CREATE TABLE class_info (
name TEXT NOT NULL,
dob TEXT,
siblings REAL,
birthplace TEXT,
yearinnyc REAL,
favoritefood TEXT
);
"""

In [58]:
cursor.execute(create_query)

### Insert the student data into the table

In [47]:
#
keys_list = []
for student in data:
    print(student['name'])
    keys = student.keys()
    for word in keys:
        if word not in keys_list:
            keys_list.append(word)
print (keys_list)

Amora Sun
Alec Ryman
Catherine Wolk
Christopher Park
Eric Landstein
Jaden
James Moody
John Americk Canque
Justin Tennenbaum
Karolina Jozefowicz
Kathrin Verestoun
Kevin Sun
Kyle Baranko
mark cleverley
Matt Oliver
Michelle Venables
Paul E. Kruger
Quan Nguyen
Rebecca
Reuben Kavalov
Sarah Smith
Taylor Appel
['name', 'DOB', 'siblings', 'Birthplace', 'yearinnyc', 'favoritefood', 'birthdate', 'yearsinnyc', 'Favoritefood', 'birthplace']


In [50]:
students_list = []
    
for student in data:
    if 'DOB' in student.keys():
        student['birthdate'] = student['DOB']
    if 'Birthplace' in student.keys():
        student['birthplace'] = student.pop('Birthplace')
    if 'Favoritefood'in student.keys():
        student['favoritefood'] =student['Favoritefood']
    if 'yearinnyc' in student.keys():
        student['yearsinnyc'] = student['yearinnyc']
    student_tuple= (student['name'], student['birthdate'],
                   student['siblings'], student['birthplace'],
                   student['yearsinnyc'], student['favoritefood'])
    students_list.append(student_tuple)

In [51]:
students_list

[('Amora Sun', '11/24/1991', 'none', 'China', '22', 'Japanese'),
 ('Alec Ryman', '2-14-1993', 1, 'New York', 5, 'sushi'),
 ('Catherine Wolk', None, 2, 'Danbury, CT', 6, 'Banh Mi'),
 ('Christopher Park', '10-26-91', 2, 'New York City, New York', 10, 'shwarma'),
 ('Eric Landstein', '01-29-91', 1, 'Long Island, NY', 3, 'Pizza'),
 ('Jaden', '08-20-1996', 1, 'Hong Kong', 22, 'Pizza'),
 ('James Moody', '02-11-1991', 2, 'London, UK', 4, 'Vegan Pizza'),
 ('John Americk Canque',
  'October 8, 1987',
  'None',
  'Manila, Philippines',
  12,
  'Steak Chimichanga'),
 ('Justin Tennenbaum', '02-21-1995', 2, 'New Brunswick, NJ', 24, 'Ribs'),
 ('Karolina Jozefowicz', '05-05-1993', 1, 'Olsztyn, Poland', 0, 'bagels'),
 ('Kathrin Verestoun', '03-14-1988', 1, 'Odessa, Ukraine', 23, 'French Fries'),
 ('Kevin Sun', '11/6/97', 1, 'New Brunswick', 0, 'Mac and Cheese'),
 ('Kyle Baranko', '09-22-95', 2, 'Tucson, AZ', 1, 'Pad Thai'),
 ('mark cleverley', 'september 9, 1997', 0, 'london', 4, 'fish sticks'),
 ('Mat

In [60]:
#stmt = '''INSERT INTO class_info (name, dob, siblings, birthplace, yearinnyc, favoritefood) VALUES (%s, %s, %s, %s, %s, %s)'''
#cursor.executemany(stmt,students_list)
#cnx.commit()

In [62]:
import pandas as pd

In [82]:
# 
cnx = mysql.connector.connect(
        host = config2.host,
        user = config2.user,
        passwd = config2.passwd,
        database= DB_NAME)
cursor = cnx.cursor()

In [83]:
cursor.execute('''SELECT name, dob FROM class_info''')

In [84]:
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]

In [85]:
df

,name,dob
0,Amora Sun,11/24/1991
1,Alec Ryman,2-14-1993
2,Catherine Wolk,None
3,Christopher Park,10-26-91
4,Eric Landstein,01-29-91
5,Jaden,08-20-1996
6,James Moody,02-11-1991
7,John Americk Canque,"October 8, 1987"
8,Justin Tennenbaum,02-21-1995
9,Karolina Jozefowicz,05-05-1993


In [102]:

df.dropna(inplace=True)
df.loc[df['dob'].str.contains('/'),'dob'] = df[df['dob'].str.contains('/')]['dob'].map(lambda x: '-'.join(x.split('/')))

In [104]:
df['dob'].str.contains

0            11-24-1991
1             2-14-1993
3              10-26-91
4              01-29-91
5            08-20-1996
6            02-11-1991
7       October 8, 1987
8            02-21-1995
9            05-05-1993
10           03-14-1988
11              11-6-97
12             09-22-95
13    september 9, 1997
14           04-18-1996
15           10-28-1993
16      January 4, 1976
17               3-2-94
18               May 10
19           06-25-1995
20           09-26-1989
21             03-26-93
Name: dob, dtype: object

### Write queries to answer the following questions:

**Questions**
- Which student was born closest to the cohort's graduation date?
- Which student has the most siblings?
- How many students are only children?
- Which 3 students have lived in NYC the shortest amount of time?
- How many students are native New Yorkers?
- Do any two students have the same favorite food?